After sucessfully loading all tables into a database and merging them into one table, this table now will be cleaned and transformed for further analysis.

But at first the required modules and credentials are loaded, and the MySQL connection is set up. 

In [1]:
import pandas as pd
import json
import mysql.connector

In [2]:
#get credentials for MySQL/dbms
with open(r'C:\Users\mathi\Desktop\Datenanalyse\Credentials\mysql.json') as json_file:
    db_credentials = json.load(json_file)['MySQL']

#credentials as variables
host='localhost'
user=db_credentials['user']
password=db_credentials['password']
database='feederwatch'

#etablish connection to the dbms
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password)

mycursor = mydb.cursor()

#select database
query=f'USE {database}'
mycursor.execute(query)

To figure out, what to do, at first the table structure is inspected.

In [3]:
query='DESCRIBE observations'
mycursor.execute(query)
data=mycursor.fetchall()
description=pd.DataFrame(data)
description.columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra']
print(description)

                 Field       Type Null  Key Default Extra
0                index  b'bigint'  YES  MUL    None      
1               LOC_ID    b'text'  YES         None      
2             LATITUDE  b'double'  YES         None      
3            LONGITUDE  b'double'  YES         None      
4    SUBNATIONAL1_CODE    b'text'  YES         None      
5      ENTRY_TECHNIQUE    b'text'  YES         None      
6               SUB_ID    b'text'  YES         None      
7               OBS_ID    b'text'  YES         None      
8                Month  b'bigint'  YES         None      
9                  Day  b'bigint'  YES         None      
10                Year  b'bigint'  YES         None      
11      PROJ_PERIOD_ID    b'text'  YES         None      
12        SPECIES_CODE    b'text'  YES         None      
13            HOW_MANY  b'bigint'  YES         None      
14           PLUS_CODE  b'bigint'  YES         None      
15               VALID  b'bigint'  YES         None      
16            

By checking the provided excel file, some differences between the current data types and the provided data types can be noticed:

| Field | current type | actual type |
|-|-|-|
| VALID | double | binary |
| REVIEWED | double | binary |
| PLUS_CODE | double | binary |
| DAY1_AM | double | binary |
| DAY1_PM | double | binary |
| DAY2_AM | double | binary |
| DAY2_PM | double | binary |

To change the data types a simple typcasting query is performed.

In [4]:
query='''ALTER TABLE observations 
        MODIFY COLUMN VALID binary,
        MODIFY COLUMN REVIEWED binary,
        MODIFY COLUMN PLUS_CODE binary,
        MODIFY COLUMN DAY1_AM binary,
        MODIFY COLUMN DAY1_PM binary,
        MODIFY COLUMN DAY2_AM binary,
        MODIFY COLUMN DAY2_PM binary
        '''
mycursor.execute(query)

Because day, month, and year are usually very interesting for analysis, it's important, that they are formated in a useful way, eg. Year should be 4 digits to avoid sorting issues.
To figure out the exact format of the columns they are fetched.

In [5]:
query='SELECT Year, month, day FROM observations LIMIT 10'
mycursor.execute(query)
data=mycursor.fetchall()
description=pd.DataFrame(data)
description.columns=['Year', 'Month', 'Day']
print(description)

   Year  Month  Day
0  2006      1   25
1  2008     11   25
2  2005     11   12
3  2008     11   25
4  2005     11   12
5  2008     11   25
6  2006      1   25
7  2005     11   12
8  2008     11   25
9  2006      1   25


According to the provided data dictonary, the column 'SUBNATIONAL1_CODE' contains a combination of country and state of the observation location.

> Format XX-XX where the left two letters indicate the country and the right two letters indicate the state or province code. E.g., CA-ON = Canada - Ontario

To utilize this information, it might be better to seperate country and region/state from each other.  
For this reason, two columns are generated:

In [6]:
query='''ALTER TABLE observations
      ADD COUNTRY VARCHAR(2),
      ADD STATE VARCHAR(2)'''

mycursor.execute(query)

In the next step, the columns should be populated, but unfortunally the folowing querry generates an error:

> DataError: 1406 (22001): Data too long for column 'STATE' at row 576571

In [ ]:
query='''UPDATE observations
      SET COUNTRY = SUBSTRING_INDEX(SUBNATIONAL1_CODE, "-", 1),
      STATE = SUBSTRING_INDEX(SUBNATIONAL1_CODE, "-", -1);
      '''
      
mycursor.execute(query)

Since the error states, that the data is to long, there might be some enteries which do not follow the xx-xx pattern. 
There are multiple ways on whats could be wrong, eg. xx-xxx and x-xxx.
To get a better idea of the data issue, the lenght of the entry and the position of the delimiter are fetched.

In [7]:
query = '''SELECT COUNT(*), INSTR(SUBNATIONAL1_CODE, "-") AS Position, CHAR_LENGTH(SUBNATIONAL1_CODE) AS LENGTH
        FROM observations 
        GROUP BY Position, LENGTH
        '''
mycursor.execute(query)
data=mycursor.fetchall()
description=pd.DataFrame(data)
description.columns=['Count', 'Position', 'Length']
print(description)

      Count  Position  Length
0  36726426         3       5
1      4566         3       3
2      2678         3       6


The position of the delimiter indicates, that all countries are two letter abbreviations, but some states/regions are not.
For futher understanding 5 entries for each length are fetched.

In [8]:
query = '''(SELECT SUBNATIONAL1_CODE, CHAR_LENGTH(SUBNATIONAL1_CODE)
        FROM observations 
        WHERE CHAR_LENGTH(SUBNATIONAL1_CODE)=3
        LIMIT 5)
        UNION ALL
        (SELECT SUBNATIONAL1_CODE, CHAR_LENGTH(SUBNATIONAL1_CODE)
        FROM observations 
        WHERE CHAR_LENGTH(SUBNATIONAL1_CODE)=5
        LIMIT 5)
        UNION ALL
        (SELECT SUBNATIONAL1_CODE, CHAR_LENGTH(SUBNATIONAL1_CODE)
        FROM observations 
        WHERE CHAR_LENGTH(SUBNATIONAL1_CODE)=6
        LIMIT 5)
        '''
mycursor.execute(query)
data=mycursor.fetchall()
description=pd.DataFrame(data)
description.columns=['SUBNATIONAL1_CODE', 'Length']
print(description)

   SUBNATIONAL1_CODE  Length
0                XX-       3
1                XX-       3
2                XX-       3
3                XX-       3
4                XX-       3
5              CA-ON       5
6              US-PA       5
7              US-MI       5
8              US-PA       5
9              US-MI       5
10            NZ-OTA       6
11            NZ-OTA       6
12            NZ-OTA       6
13            NZ-OTA       6
14            NZ-OTA       6


This shows, that there are some empty state/region codes (xx-) and some three letter codes. Because of this the columns character limit should be increased.

In [9]:
query = '''ALTER TABLE observations
        MODIFY STATE VARCHAR(3);
        '''
        
mycursor.execute(query)

Now the querry from above can be re-runed. Additionally the empty values can be replaced.

In [10]:
query='''UPDATE observations
      SET COUNTRY = SUBSTRING_INDEX(SUBNATIONAL1_CODE, "-", 1),
      STATE = SUBSTRING_INDEX(SUBNATIONAL1_CODE, "-", -1);
      '''

mycursor.execute(query)

query='''UPDATE observations
      SET STATE = "XX"
      WHERE STATE IS NULL OR STATE = "";
      '''
      
mycursor.execute(query)

And as control the lengths of  SUBNATIONAL1_CODE and STATE can be checked.

In [11]:
query = '''SELECT COUNT(*), CHAR_LENGTH(SUBNATIONAL1_CODE) AS LENGTH_SUBNATIONAL, CHAR_LENGTH(STATE) AS LENGTH_STATE
        FROM observations 
        GROUP BY LENGTH_SUBNATIONAL, LENGTH_STATE
        '''
mycursor.execute(query)
data=mycursor.fetchall()
description=pd.DataFrame(data)
description.columns=['Count', 'Length_SUBNATIONAL', 'Length_STATE']
print(description)

      Count  Length_SUBNATIONAL  Length_STATE
0  36726426                   5             2
1      4566                   3             2
2      2678                   6             3


To check the countries for any irregularities all distinct countries and the states per contry are inspected.

In [12]:
query='''SELECT 
  COUNTRY, COUNT(DISTINCT(STATE))
  FROM observations
  GROUP BY COUNTRY
  '''

mycursor.execute(query)
data=mycursor.fetchall()
country_state=pd.DataFrame(data)
country_state.columns=['Country', 'State_Count']
print(country_state)

  Country  State_Count
0      CA           13
1      MX            2
2      NZ            1
3      PM            1
4      US           51
5      XX            1


As Canada has 10 provinces and three territories, and the USA has 50 states, the numbers seem to be correct. Note that the data might contain a unknown state labled "XX".
Though the dataset contains only two out of 31 states from Mexico, this might be fine, because the focus of this project is on the USA and Canada.
So it might be the simplesst way to remove every other country, but before this can be done, its importent to know, how much data would be lost.

In [13]:
query='''SELECT 
  COUNTRY, COUNT(*)
  FROM observations
  GROUP BY COUNTRY
  '''

mycursor.execute(query)
data=mycursor.fetchall()
country_entries=pd.DataFrame(data)
country_entries.columns=['Country', 'Count']
print(country_entries)

  Country     Count
0      CA   4686465
1      MX      2595
2      NZ        83
3      PM      1830
4      US  32039961
5      XX      2736


Because the other contries make only a small fraction of the overall data, they can be removed withaout large issues.

In [14]:
query = '''DELETE FROM observations 
        WHERE COUNTRY<>"CA" AND COUNTRY<>"US";
        '''
        
mycursor.execute(query)

query='''SELECT 
  COUNTRY, COUNT(*)
  FROM observations
  GROUP BY COUNTRY
  '''

mycursor.execute(query)
data=mycursor.fetchall()
country_entries=pd.DataFrame(data)
country_entries.columns=['Country', 'Count']
print(country_entries)

  Country     Count
0      CA   4686465
1      US  32039961
